# *Classificador*

- Download do modelo em Portugues

In [ ]:
!python -m spacy download pt_core_news_sm

     |████████████████████████████████| 21.2MB 1.2MB/s 
  Created wheel for pt-core-news-sm: filename=pt_core_news_sm-2.2.5-cp36-none-any.whl size=21186283 sha256=075f2e9365e89fb6a9973e9c22d26c03e0f790d29f030cf2ad9e7d69072599e2
  Stored in directory: /tmp/pip-ephem-wheel-cache-sb0k2_jp/wheels/ea/94/74/ec9be8418e9231b471be5dc7e1b45dd670019a376a6b5bc1c0
Successfully built pt-core-news-sm
✔ Download and installation successful
You can now load the model via spacy.load('pt_core_news_sm')


Vetorização de texto

- recebe um texto e retorna uma lista de token. 
- criar uma vetorização indivisual

In [ ]:
import spacy
import numpy as np
import pandas as pd
from gensim.models import KeyedVectors

w2v_modelo_cbow = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/word2ver/modelo_cbow.txt")
w2v_modelo_sg = KeyedVectors.load_word2vec_format("/content/drive/MyDrive/word2ver/modelo_skipgram.txt")

artigo_treino= pd.read_csv("/content/drive/MyDrive/word2ver/treino.csv")
artigo_teste= pd.read_csv("/content/drive/MyDrive/word2ver/teste.csv")



## Pré-processamento

Remove os stop words e caracteres não alfabéticos e deixando palavras em minúsculo, retornando exatamente o que foi solicitado.

In [ ]:
# pipeline

nlp = spacy.load("pt_core_news_sm", disable=["paser", "ner", "tagger", "textcat"]) # não é necessário alguns pré-processamentos

def tokenizador(texto): #função adaptada para tokinizador
  
  doc = nlp(texto)
  tokens_validos = []
  for token in doc:
    e_valido = not token.is_stop and token.is_alpha # verifica se o token é valido 
    if e_valido:
      tokens_validos.append(token.text.lower())

  return tokens_validos


texto = "Rio de Janeiro 1212122 c****é uma cidade maravilhosa!" # teste

tokens = tokenizador(texto)
print(tokens)

['rio', 'janeiro', 'cidade', 'maravilhosa']


Combinação de vetorores

In [ ]:
def combinacao_de_vetores_por_soma(palavras, modelo):

    vetor_resultante = np.zeros(300)

    for pn in palavras:
        try:
            vetor_resultante += modelo.get_vector(pn)

        except KeyError:
            pass # errou bola pra frente rs

    return vetor_resultante

#teste
vetor_texto = combinacao_de_vetores_por_soma(tokens, w2v_modelo_cbow)
print(vetor_texto.shape)
print(vetor_texto)

(300,)
[ 2.48783135e+00  6.93557234e-01  1.47610006e+00 -5.23369528e-01
  2.85388979e+00  2.47088224e+00 -1.05546229e+00 -2.66558081e-01
  2.15231224e+00 -1.99439583e+00 -2.31663816e+00 -4.98498887e-01
  1.89665854e-01  1.03707394e+00 -1.07846737e+00  3.25014181e-01
 -4.01880398e-01 -7.08095433e-01 -8.95570785e-01 -2.81728798e+00
 -1.70187348e+00  3.40880272e+00 -6.24393374e-02 -1.99704210e+00
 -2.28661652e+00  2.53879085e-01  2.45454830e+00 -2.36154789e+00
 -1.08460143e+00  3.17483291e-01 -1.33512139e+00  5.04100740e-01
 -6.91189736e-01  6.49076641e-01  2.28803528e+00 -8.15818250e-01
  1.46883067e+00  1.09723333e+00 -8.42743069e-01  1.57012936e-01
  9.64347839e-01  1.36279438e+00  1.76930012e+00  9.29067584e-01
  3.73866349e+00  1.71907419e+00  1.25131183e+00 -1.75918967e-01
  2.81676245e+00 -2.72921652e-01  1.67302340e-01 -9.19192145e-01
 -3.83567190e+00 -1.63536172e+00  5.61754334e-01 -4.13346913e-01
 -9.96637508e-01  5.13449609e-02 -1.38834625e+00 -4.02037070e-01
 -6.74634829e-01  

Vetorização

In [ ]:
def matriz_vetores(textos, modelo):
    x = len(textos)
    y = 300 #colunas/dimensões
    matriz = np.zeros((x,y))

    for i in range(x):
        palavras = tokenizador(textos.iloc[i])
        matriz[i] = combinacao_de_vetores_por_soma(palavras, modelo)

    return matriz

matriz_vetores_treino_cbow = matriz_vetores(artigo_treino.title, w2v_modelo_cbow)
matriz_vetores_teste_cbow = matriz_vetores(artigo_teste.title, w2v_modelo_cbow)
print(matriz_vetores_treino_cbow.shape)
print(matriz_vetores_teste_cbow.shape)

(90000, 300)
(20513, 300)


Classificação



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

def classificador(modelo, x_treino, y_treino, x_teste, y_teste):

      RL = LogisticRegression(max_iter = 800)
      RL.fit(x_treino, y_treino)
      categorias = RL.predict(x_teste)
      resultados = classification_report(y_teste, categorias)
      print(resultados)

      return RL

RL_cbow = classificador(w2v_modelo_cbow,
                        matriz_vetores_treino_cbow,
                        artigo_treino.category,
                        matriz_vetores_teste_cbow,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.63      0.80      0.71      1698
     esporte       0.93      0.86      0.89      4663
   ilustrada       0.13      0.85      0.23       131
     mercado       0.84      0.78      0.81      5867
       mundo       0.74      0.83      0.78      2051

    accuracy                           0.79     20513
   macro avg       0.68      0.81      0.69     20513
weighted avg       0.82      0.79      0.80     20513



In [ ]:
matriz_vetores_treino_sg = matriz_vetores(artigo_treino.title, w2v_modelo_sg)
matriz_vetores_teste_sg = matriz_vetores(artigo_teste.title, w2v_modelo_sg)

RL_sg = classificador(w2v_modelo_sg,
                        matriz_vetores_treino_sg,
                        artigo_treino.category,
                        matriz_vetores_teste_sg,
                        artigo_teste.category)

              precision    recall  f1-score   support

     colunas       0.81      0.71      0.76      6103
   cotidiano       0.64      0.81      0.72      1698
     esporte       0.94      0.88      0.90      4663
   ilustrada       0.14      0.89      0.25       131
     mercado       0.84      0.79      0.82      5867
       mundo       0.75      0.84      0.79      2051

    accuracy                           0.79     20513
   macro avg       0.69      0.82      0.71     20513
weighted avg       0.82      0.79      0.80     20513



Importanto o modelo

In [ ]:
import pickle

with open("/content/drive/MyDrive/word2ver/rl_cbow.pkl","wb") as f:
  pickle.dump(RL_cbow, f)


In [ ]:
with open("/content/drive/MyDrive/word2ver/rl_sg.pkl","wb") as f:
  pickle.dump(RL_sg, f)